# Задание 1 (5 балла)

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [ ]:
# скачиваем корпус
!mkdir data
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_wsd_50k.txt.zip -P data
!unzip -o data/corpus_wsd_50k.txt.zip -d data/

--2024-10-01 15:09:29--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_wsd_50k.txt.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_wsd_50k.txt.zip [following]
--2024-10-01 15:09:30--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_wsd_50k.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4723095 (4.5M) [application/zip]
Saving to: ‘data/corpus_wsd_50k.txt.zip’

corpus_wsd_50k.txt. 100%[===================>]   4.50M  --.-KB/s    in 0.06s   

2024-10-01 15:09:30 (

In [ ]:
# вычитываем корпус в память
corpus = open('data/corpus_wsd_50k.txt').read()
print(corpus[:200])

	how	How
long%3:00:02::	long	long
	have	has
	it	it
be%2:42:03::	be	been
	since	since
	you	you
review%2:31:00::	review	reviewed
	the	the
objective%1:09:00::	objective	objectives
	of	of
	you	your
benefi


In [ ]:
# парсим корпус так, чтобы в итоге он представлял собой список с предложениями,
# преложения были бы списками со словами,
# а слова были бы списками из трех элементов -- значения (если слово не однозначно), леммы и самого слова, как оно употреблено в предложении
corpus_wsd = []
corpus = corpus.split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])
corpus_wsd[1]

[['', 'have', 'Have'],
 ['', 'you', 'you'],
 ['permit%2:41:00::', 'permit', 'permitted'],
 ['', 'it', 'it'],
 ['', 'to', 'to'],
 ['become%2:42:01::', 'become', 'become'],
 ['', 'a', 'a'],
 ['giveaway%1:21:00::', 'giveaway', 'giveaway'],
 ['program%1:09:01::', 'program', 'program'],
 ['rather%4:02:02::', 'rather', 'rather'],
 ['', 'than', 'than'],
 ['', 'one', 'one'],
 ['', 'that', 'that'],
 ['have%2:42:00::', 'have', 'has'],
 ['', 'the', 'the'],
 ['goal%1:09:00::', 'goal', 'goal'],
 ['', 'of', 'of'],
 ['improved%3:00:00::', 'improved', 'improved'],
 ['employee%1:18:00::', 'employee', 'employee'],
 ['morale%1:26:00::', 'morale', 'morale'],
 ['', 'and', 'and'],
 ['', ',', ','],
 ['consequently%4:02:00::', 'consequently', 'consequently'],
 ['', ',', ','],
 ['increased%3:00:00::', 'increased', 'increased'],
 ['productivity%1:07:00::', 'productivity', 'productivity'],
 ['', '?', '?']]

In [ ]:
# !pip install nltk -U

In [ ]:
# импортируем nltk
import nltk

# импортируем wordnet, чтобы с помощью него искать определения словам
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# выгружаем список стоп-слов, чтобы убирать их из предложений
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = set(stopwords.words())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import string

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
nltk.download('punkt_tab')
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize

from typing import Union

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def sentence2bow(sent:Union[str,list]) -> list:
  """
  Превращает предложение в виде строки или список слов
  в список лемм (изначально без стоп-слов и пунктуации)
  """
  res = []
  if isinstance(sent, str):
    sent = word_tokenize(sent.lower())
  for token in sent:
    # if token not in stopwords and token not in string.punctuation:
    res.append(token.lower())
  return res

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


### Жаккар

In [ ]:
# функция для подбора наиболее подходящего значения на основе количества слов,
# использованных в определении и предложении.
# Если общих слов нет, то возвращается Null

def lesk_jaccard(word, context:list):
  max_jaccard_score = 0
  best_jaccard_key = None

  for synset in wn.synsets(word):
    word_key = synset.lemmas()[0].key()
    word_definition = sentence2bow(synset.definition())

    intersection = (set(word_definition) & set(context))
    union = (set(word_definition) | set(context))
    jaccard_score =  len(intersection) / len(union)

    if jaccard_score > max_jaccard_score:
      max_jaccard_score = jaccard_score
      best_jaccard_key = word_key

    return best_jaccard_key

In [ ]:
# вычисляем точность алгоритма lesk_jaccard

total = 0  # общее количество неоднозначных слов
correct = 0  # количество слов, для которых правильно подобраны значения

for doc in corpus_wsd:
  if doc != [['']]:
    sent = sentence2bow([t[1] for t in doc])
    for w in doc:
      if w[0] != '': # если слово не однозначно
        correct += (lesk_jaccard(w[1].lower(), sent) == w[0]) * 1  # если подобранное алгоритмом значение совпадает со значением, указанным вкорпусе, то к correct прибавляется 1
        total += 1  # увеличиваем количество обработанных неоднозначных слов

accuracy = correct / total

print('Слов рассмотрено: ' + '{:,}'.format(total),
      'Из них правильно определено: ' + '{:,}'.format(correct),
      'Accuracy: ' + "{:.2%}".format(accuracy),
      sep = '\n'
)

Слов рассмотрено: 239,913
Из них правильно определено: 61,914
Accuracy: 25.81%


Эксперименты с предобработкой предложений из корпуса и определений дали следующие результаты:

- токенизация определений с помощью word_tokenize(), с очисткой от стоп-слов, приведением к нижнему регистру и очисткой от пунктуации -- 2.6%
- без какой-либо обработки -- только split(), предложения из корпуса составляются из [1] элементов -- 25.24%
- без какой-либо обработки -- только split(), предложения из корпуса составляются из [2] элементов -- 23.61%
- токенизация определений с помощью word_tokenize(), очистка от пунктуации и приведение к нижнему регистру, предложения из корпуса составляются из [2] элементов -- 24.26%
- ЛУЧШИЙ РЕЗУЛЬТАТ: только токенизация определений с помощью word_tokenize() и приведение к нижнему регистру, предложения из корпуса составляются из [1] элементов -- 25.81%

### Косинусное расстояние

In [ ]:
!python -m pip install torch torchvision torchaudio
!python -m pip install sentence_transformers transformers accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 62.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [7]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# функция для подбора наиболее подходящего значения на основе близости векторов текстов

def lesk_cosine(word, embedded_context):
    max_similarity_score = 0
    best_key = None

    word_keys = []
    definition_embedings = []

    for synset in wn.synsets(word):
      word_keys.append(synset.lemmas()[0].key())

      definition_embedings.append(embed(synset.definition()))

    similarity_score = cosine_similarity(embedded_context, definition_embedings)[0]

    pairs = list(zip(word_keys, similarity_score))

    for p in pairs:
      if p[1] > max_similarity_score:
        max_similarity_score = p[1]
        best_key = p[0]

    return best_key

In [ ]:
# возьмем рандомную выборку из 5 тыс. предложений, чтобы не ждать >10 часов для подсчитывания эмбедов всего корпуса
import random
corpus_wsd_5k =random.sample(corpus_wsd, 5_000)

*забыла указать random seed

In [ ]:
# вычисляем точность алгоритма lesk_cosine

from tqdm import tqdm

total = 0  # общее количество неоднозначных слов
correct = 0  # количество слов, для которых правильно подобраны значения

for doc in tqdm(corpus_wsd_5k):
  if doc != [['']]:
    sent = ' '.join([t[2] for t in doc])
    embedded_sent = embed(sent).reshape(1, -1)
    for w in doc:
      if w[0] != '': # если слово не однозначно
        try:
          correct += (lesk_cosine(w[1].lower(), embedded_sent) == w[0]) * 1  # если подобранное алгоритмом значение совпадает со значением, указанным вкорпусе, то к correct прибавляется 1
          total += 1  # увеличиваем количество обработанных неоднозначных слов
        except Exception as e:
          print(e)
          print(w)
          print(doc, '\n')

accuracy = correct / total

print('')
print('Слов рассмотрено: ' + '{:,}'.format(total),
      'Из них правильно определено: ' + '{:,}'.format(correct),
      'Accuracy: ' + "{:.2%}".format(accuracy),
      sep = '\n'
)

100%|██████████| 5000/5000 [57:00<00:00,  1.46it/s]


Слов рассмотрено: 24,823
Из них правильно определено: 6,615
Accuracy: 26.65%


Кажется, что оба алгоритма работают одинаково хорошо (или одинаково плохо) с accuracy 25%-26%. При этом стоит уточнить, что первый алгоритм мы проверяли на полном корпусе -- 50 тыс. предложениях, а второй алгоритм -- только на 5 тыс. предложениях.

# Задание 2 (5 балла)
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/nlpub/russe-wsi-kit/initial/data/main/wiki-wiki/train.csv', sep='\t')
df

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,замок,1,NaN,"0-5, 339-344",замок владимира мономаха в любече . многочисле...
1,2,замок,1,NaN,"11-16, 17-22, 188-193","шильонский замок замок шильйон ( ) , известный..."
2,3,замок,1,NaN,299-304,проведения архитектурно - археологических рабо...
3,4,замок,1,NaN,111-116,"топи с . , л . белокуров легенда о завещании м..."
4,5,замок,1,NaN,"134-139, 262-267",великий князь литовский гедимин после успешной...
...,...,...,...,...,...,...
434,435,бор,2,NaN,10-13,ленточный бор ле́нточные бо́ры — сосновые трав...
435,436,бор,2,NaN,"101-104, 149-152, 207-210, 259-262, 352-355, 4...","в окрестностях барнаула , составляет — км . н..."
436,437,бор,2,NaN,"17-20, 138-141, 262-265, 310-313",также в сосновом бору открыта секция биатлона ...
437,438,бор,2,NaN,183-186,"экспресс банк , мособлбанк , внешпромбанк , ба..."


In [3]:
df['word'].value_counts()

,count
word,
замок,138
суда,135
лук,110
бор,56


In [4]:
df.groupby(['word', 'gold_sense_id'])['word'].count()

word   gold_sense_id
бор    1                 14
       2                 42
замок  1                100
       2                 38
лук    1                 65
       2                 45
суда   1                100
       2                 35
Name: word, dtype: int64

In [5]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation
import numpy as np
from sklearn.metrics import adjusted_rand_score
import seaborn as sns

In [6]:
!python -m pip install torch torchvision torchaudio
!python -m pip install sentence_transformers transformers accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 110.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [7]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
# embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
df['embed'] = df['context'].apply(embed)
df.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context,embed
0,1,замок,1,NaN,"0-5, 339-344",замок владимира мономаха в любече . многочисле...,"[0.046547443, -0.0072870273, -0.0017295851, -0..."
1,2,замок,1,NaN,"11-16, 17-22, 188-193","шильонский замок замок шильйон ( ) , известный...","[0.03495271, -0.06329054, 0.034697242, -0.0247..."
2,3,замок,1,NaN,299-304,проведения архитектурно - археологических рабо...,"[0.024540573, -0.02033471, 0.011279049, -0.056..."
3,4,замок,1,NaN,111-116,"топи с . , л . белокуров легенда о завещании м...","[0.055886272, -0.042413253, 0.0068183937, -0.0..."
4,5,замок,1,NaN,"134-139, 262-267",великий князь литовский гедимин после успешной...,"[0.06316372, -0.025873901, 0.014728018, -0.008..."


### DBSCAN

In [65]:
def cluster_dbscan(data, true_labels, epsilon, min_samples):
  ARI = []
  db = DBSCAN(eps=epsilon, min_samples=min_samples).fit(data)
  predicted_labels = db.labels_

  # Number of clusters in labels, ignoring noise if present.
  n_clusters_ = len(set(predicted_labels)) - (1 if -1 in predicted_labels else 0)
  n_noise_ = list(predicted_labels).count(-1)

  # расчитываем метрику для отдельного слова
  ARI.append(adjusted_rand_score(true_labels, predicted_labels))

  print("\tEstimated number of clusters: %d" % n_clusters_)
  print("\tEstimated number of noise points: %d" % n_noise_)
  print('\tSentences total: ', len(data))
  print('\tMean ARI: ', round(np.mean(ARI), 3))


In [58]:
# @title Эксперимент 1
EPSILON = 0.3 # @param {type:"slider", min:0, max:1, step:0.1}
MIN_SAMPLES = 10 # @param {type:"slider", min:2, max:100, step:2}

for w in df['word'].unique():
  df_group = df[df['word']==w]
  print(w)
  cluster_dbscan(df_group['embed'].to_list(), df_group['gold_sense_id'].to_list(), epsilon=EPSILON, min_samples=MIN_SAMPLES)


замок
	Estimated number of clusters: 0
	Estimated number of noise points: 138
	Sentences total:  138
	Mean ARI:  0.0
лук
	Estimated number of clusters: 0
	Estimated number of noise points: 110
	Sentences total:  110
	Mean ARI:  0.0
суда
	Estimated number of clusters: 0
	Estimated number of noise points: 135
	Sentences total:  135
	Mean ARI:  0.0
бор
	Estimated number of clusters: 0
	Estimated number of noise points: 56
	Sentences total:  56
	Mean ARI:  0.0


In [59]:
# @title Эксперимент 2
EPSILON = 0.5 # @param {type:"slider", min:0, max:1, step:0.1}
MIN_SAMPLES = 6 # @param {type:"slider", min:2, max:100, step:2}

for w in df['word'].unique():
  df_group = df[df['word']==w]
  print(w)
  cluster_dbscan(df_group['embed'].to_list(), df_group['gold_sense_id'].to_list(), epsilon=EPSILON, min_samples=MIN_SAMPLES)

замок
	Estimated number of clusters: 1
	Estimated number of noise points: 53
	Sentences total:  138
	Mean ARI:  0.005
лук
	Estimated number of clusters: 2
	Estimated number of noise points: 70
	Sentences total:  110
	Mean ARI:  0.02
суда
	Estimated number of clusters: 2
	Estimated number of noise points: 71
	Sentences total:  135
	Mean ARI:  0.019
бор
	Estimated number of clusters: 1
	Estimated number of noise points: 49
	Sentences total:  56
	Mean ARI:  -0.099


In [61]:
# @title Эксперимент 3
EPSILON = 0.8 # @param {type:"slider", min:0, max:1, step:0.1}
MIN_SAMPLES = 4 # @param {type:"slider", min:2, max:100, step:2}

for w in df['word'].unique():
  df_group = df[df['word']==w]
  print(w)
  cluster_dbscan(df_group['embed'].to_list(), df_group['gold_sense_id'].to_list(), epsilon=EPSILON, min_samples=MIN_SAMPLES)

замок
	Estimated number of clusters: 1
	Estimated number of noise points: 1
	Sentences total:  138
	Mean ARI:  -0.009
лук
	Estimated number of clusters: 1
	Estimated number of noise points: 2
	Sentences total:  110
	Mean ARI:  0.002
суда
	Estimated number of clusters: 1
	Estimated number of noise points: 0
	Sentences total:  135
	Mean ARI:  0.0
бор
	Estimated number of clusters: 1
	Estimated number of noise points: 0
	Sentences total:  56
	Mean ARI:  0.0


In [62]:
# @title Эксперимент 4
EPSILON = 0.4 # @param {type:"slider", min:0, max:1, step:0.1}
MIN_SAMPLES = 4 # @param {type:"slider", min:2, max:100, step:2}

for w in df['word'].unique():
  df_group = df[df['word']==w]
  print(w)
  cluster_dbscan(df_group['embed'].to_list(), df_group['gold_sense_id'].to_list(), epsilon=EPSILON, min_samples=MIN_SAMPLES)

замок
	Estimated number of clusters: 4
	Estimated number of noise points: 119
	Sentences total:  138
	Mean ARI:  -0.024
лук
	Estimated number of clusters: 3
	Estimated number of noise points: 96
	Sentences total:  110
	Mean ARI:  -0.011
суда
	Estimated number of clusters: 6
	Estimated number of noise points: 90
	Sentences total:  135
	Mean ARI:  -0.064
бор
	Estimated number of clusters: 1
	Estimated number of noise points: 52
	Sentences total:  56
	Mean ARI:  -0.074


In [67]:
# %%timeit
# @title Эксперимент 5
EPSILON = 0.6 # @param {type:"slider", min:0, max:1, step:0.1}
MIN_SAMPLES = 4 # @param {type:"slider", min:2, max:100, step:2}

for w in df['word'].unique():
  df_group = df[df['word']==w]
  print(w)
  cluster_dbscan(df_group['embed'].to_list(), df_group['gold_sense_id'].to_list(), epsilon=EPSILON, min_samples=MIN_SAMPLES)


замок
	Estimated number of clusters: 1
	Estimated number of noise points: 15
	Sentences total:  138
	Mean ARI:  0.102
лук
	Estimated number of clusters: 2
	Estimated number of noise points: 13
	Sentences total:  110
	Mean ARI:  0.073
суда
	Estimated number of clusters: 1
	Estimated number of noise points: 18
	Sentences total:  135
	Mean ARI:  0.153
бор
	Estimated number of clusters: 2
	Estimated number of noise points: 5
	Sentences total:  56
	Mean ARI:  0.066


### Выводы о DBSCAN

Качество кластеризации: удачные параметры подобрать не удалось

Скорость работы: результаты timeit для последнего эксперимента 25.7 ms ± 687 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Интуитивность параметров: не очевидны, но понять можно

### Affinity Propagation

In [79]:
clustering = AffinityPropagation(random_state=5).fit(X)

In [85]:
def cluster_affprop(data, true_labels, damping):
  ARI = []
  db = AffinityPropagation(damping=damping).fit(data)
  predicted_labels = db.labels_

  # Number of clusters in labels, ignoring noise if present.
  n_clusters_ = len(set(predicted_labels)) - (1 if -1 in predicted_labels else 0)
  n_noise_ = list(predicted_labels).count(-1)

  # расчитываем метрику для отдельного слова
  ARI.append(adjusted_rand_score(true_labels, predicted_labels))

  print("\tEstimated number of clusters: %d" % n_clusters_)
  print("\tEstimated number of noise points: %d" % n_noise_)
  print('\tSentences total: ', len(data))
  print('\tMean ARI: ', round(np.mean(ARI), 3))


In [86]:
# @title Эксперимент 1
DAMPING = 0.5 # @param {type:"slider", min:0.5, max:0.9, step:0.1}

for w in df['word'].unique():
  df_group = df[df['word']==w]
  print(w)
  cluster_affprop(df_group['embed'].to_list(), df_group['gold_sense_id'].to_list(), DAMPING)


замок
	Estimated number of clusters: 28
	Estimated number of noise points: 0
	Sentences total:  138
	Mean ARI:  0.052
лук
	Estimated number of clusters: 22
	Estimated number of noise points: 0
	Sentences total:  110
	Mean ARI:  0.035
суда
	Estimated number of clusters: 26
	Estimated number of noise points: 0
	Sentences total:  135
	Mean ARI:  0.03
бор
	Estimated number of clusters: 12
	Estimated number of noise points: 0
	Sentences total:  56
	Mean ARI:  0.054


In [87]:
# @title Эксперимент 2
DAMPING = 0.9 # @param {type:"slider", min:0.5, max:0.9, step:0.1}

for w in df['word'].unique():
  df_group = df[df['word']==w]
  print(w)
  cluster_affprop(df_group['embed'].to_list(), df_group['gold_sense_id'].to_list(), DAMPING)


замок
	Estimated number of clusters: 28
	Estimated number of noise points: 0
	Sentences total:  138
	Mean ARI:  0.052
лук
	Estimated number of clusters: 4
	Estimated number of noise points: 0
	Sentences total:  110
	Mean ARI:  0.082
суда
	Estimated number of clusters: 25
	Estimated number of noise points: 0
	Sentences total:  135
	Mean ARI:  0.028
бор
	Estimated number of clusters: 13
	Estimated number of noise points: 0
	Sentences total:  56
	Mean ARI:  0.05


In [88]:
# @title Эксперимент 3
DAMPING = 0.7 # @param {type:"slider", min:0.5, max:0.9, step:0.1}

for w in df['word'].unique():
  df_group = df[df['word']==w]
  print(w)
  cluster_affprop(df_group['embed'].to_list(), df_group['gold_sense_id'].to_list(), DAMPING)


замок
	Estimated number of clusters: 28
	Estimated number of noise points: 0
	Sentences total:  138
	Mean ARI:  0.052
лук
	Estimated number of clusters: 22
	Estimated number of noise points: 0
	Sentences total:  110
	Mean ARI:  0.035
суда
	Estimated number of clusters: 26
	Estimated number of noise points: 0
	Sentences total:  135
	Mean ARI:  0.03
бор
	Estimated number of clusters: 13
	Estimated number of noise points: 0
	Sentences total:  56
	Mean ARI:  0.05


In [90]:
# @title Эксперимент 4
DAMPING = 0.8 # @param {type:"slider", min:0.5, max:0.9, step:0.1}

for w in df['word'].unique():
  df_group = df[df['word']==w]
  print(w)
  cluster_affprop(df_group['embed'].to_list(), df_group['gold_sense_id'].to_list(), DAMPING)


замок
	Estimated number of clusters: 28
	Estimated number of noise points: 0
	Sentences total:  138
	Mean ARI:  0.052
лук
	Estimated number of clusters: 22
	Estimated number of noise points: 0
	Sentences total:  110
	Mean ARI:  0.035
суда
	Estimated number of clusters: 26
	Estimated number of noise points: 0
	Sentences total:  135
	Mean ARI:  0.03
бор
	Estimated number of clusters: 13
	Estimated number of noise points: 0
	Sentences total:  56
	Mean ARI:  0.05


In [91]:
# @title Эксперимент 5
DAMPING = 0.85 # @param {type:"slider", min:0.5, max:0.9, step:0.1}

for w in df['word'].unique():
  df_group = df[df['word']==w]
  print(w)
  cluster_affprop(df_group['embed'].to_list(), df_group['gold_sense_id'].to_list(), DAMPING)


замок
	Estimated number of clusters: 28
	Estimated number of noise points: 0
	Sentences total:  138
	Mean ARI:  0.052
лук
	Estimated number of clusters: 22
	Estimated number of noise points: 0
	Sentences total:  110
	Mean ARI:  0.035
суда
	Estimated number of clusters: 25
	Estimated number of noise points: 0
	Sentences total:  135
	Mean ARI:  0.028
бор
	Estimated number of clusters: 13
	Estimated number of noise points: 0
	Sentences total:  56
	Mean ARI:  0.05


In [96]:
# @title Эксперимент 6
DAMPING = 0.95 # @param {type:"slider", min:0.5, max:0.9999, step:0.1}

for w in df['word'].unique():
  df_group = df[df['word']==w]
  print(w)
  cluster_affprop(df_group['embed'].to_list(), df_group['gold_sense_id'].to_list(), DAMPING)


замок
	Estimated number of clusters: 3
	Estimated number of noise points: 0
	Sentences total:  138
	Mean ARI:  0.003
лук
	Estimated number of clusters: 4
	Estimated number of noise points: 0
	Sentences total:  110
	Mean ARI:  0.082
суда
	Estimated number of clusters: 29
	Estimated number of noise points: 0
	Sentences total:  135
	Mean ARI:  0.03
бор
	Estimated number of clusters: 12
	Estimated number of noise points: 0
	Sentences total:  56
	Mean ARI:  0.083


### Agglomerative clustering

### Kmeans

Agglomerative clustering и Kmeans не успела сделать :(